# Loading the [Python Data Science Handbook]() as modules with `importnb` 

In [1]:
from importnb import Partial, Notebook
from pathlib import Path
from warnings import warn

# Clone the [Python Data Science from Github](https://github.com/jakevdp/PythonDataScienceHandbook)

1. Glob over all the notebooks.
2. Use the Partial loader - permit broken notebooks to load, but stop there - to import the notebooks from the book as Python modules.
    
    Display the error or warning during each load.
    
3. Exceptions and output are on the `__exception__` and `__output__` attributes, respectively.

In [2]:
repo = None
notebooks = ''
display=True
timeout=2

__doc__ = (__doc__ or "") + """
Benchmark the importability of Github projects with notebooks in them.

---
    
- repo: "https://www.github.com/jakevdp/PythonDataScienceHandbook"
  notebooks: "PythonDataScienceHandbook/notebooks/"

- repo: "https://github.com/AllenDowney/ThinkDSP"
  notebooks: "ThinkDSP/code"
"""

notebooks = Path(notebooks)

In [3]:
    def unlink():
        if 'ThinkDSP' in str(notebooks):
            for link in Path().glob('*'):
                if link.suffix != '.ipynb': 
                    if link.is_symlink(): Path(link.name).unlink()

In [4]:
class Timed(Partial):
    def exec_module(self, module):
        try:
            __import__('timeout_decorator').timeout(timeout)(super().exec_module)(module)
        except BaseException as e: 
            module.__exception__ = e
        return module
        

In [5]:
if repo:
    if not notebooks.exists():
        from IPython import get_ipython
        !git clone $repo --depth 1
        
    unlink()
    if 'ThinkDSP' in str(notebooks):
        for link in list(notebooks.glob('*')):
            if link.suffix != '.ipynb':
                if link.is_file(): Path(link.name).symlink_to(link)

In [6]:
if repo and __name__ != '__main__':
    modules = {}
    for path in notebooks.glob('*.ipynb'):
        modules[path] = module = Timed.from_filename(path, stdout=True, stderr=True, display=display)
        print(f"{'🏆😩'[bool(module.__exception__)]} {path}")
        

In [7]:
if repo and __name__ != '__main__':
    from pandas import DataFrame, set_option, concat
    set_option('precision', 1)

    df = DataFrame({
        path: {
            k: getattr(module, k) for k in dir(module)
        } for path, module in modules.items()
    }).T
    concat({'Module Attributes': df}, axis=1).sample(2)

In [8]:
if repo and __name__ != '__main__':
    print((
        "%2.1f{} of the %s notebooks loaded with a import threshold of %f seconds"
        % (df['__exception__'].isnull().mean().__mul__(100), notebooks.parts[0], timeout)
    ).format('%'))

In [9]:
unlink()